In [1]:
import pandas as pd
import numpy as np

In [2]:
# Connect Colab with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load data
airports = pd.read_csv("/content/drive/MyDrive/DATA 6560/Data/Raw datasets/airports.csv")
routes = pd.read_csv("/content/drive/MyDrive/DATA 6560/Data/Raw datasets/routes.csv")

# Select relevant columns
airports = airports[["Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude"]]

routes = routes[["Source Airport", "Destination Airport", "Source Airport ID", "Destination Airport ID"]]

In [4]:
# Merge source airport info
routes = routes.merge(
    airports[["IATA", "Country"]].rename(columns={"IATA": "Source Airport", "Country": "Source Country"}),
    on="Source Airport",
    how="left"
)

# Merge destination airport info
routes = routes.merge(
    airports[["IATA", "Country"]].rename(columns={"IATA": "Destination Airport", "Country": "Destination Country"}),
    on="Destination Airport",
    how="left"
)


In [5]:
#3. Compute country-level metrics
#A — Airport count
airport_count = (
    airports.groupby("Country")
            .size()
            .rename("airport_count")
            .reset_index()
)

#B — Total routes per country
routes_by_country = (
    routes.groupby("Source Country")
          .size()
          .rename("total_routes")
          .reset_index()
          .rename(columns={"Source Country": "Country"})
)

#C — International reach (unique destination countries)
international_reach = (
    routes[routes["Source Country"] != routes["Destination Country"]]
    .groupby("Source Country")["Destination Country"]
    .nunique()
    .rename("unique_dest_countries")
    .reset_index()
    .rename(columns={"Source Country": "Country"})
)

#D — Hub strength - Largest airport’s route share within each country.

# Routes per airport
routes_per_airport = (
    routes.groupby(["Source Country", "Source Airport"])
          .size()
          .rename("routes_from_airport")
          .reset_index()
)

# Compute hub strength
hub_strength = (
    routes_per_airport.groupby("Source Country").apply(
        lambda df: df["routes_from_airport"].max() / df["routes_from_airport"].sum()
    )
    .rename("hub_strength")
    .reset_index()
    .rename(columns={"Source Country": "Country"})
)

/tmp/ipython-input-2426207598.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  routes_per_airport.groupby("Source Country").apply(


In [6]:
# Combine metrics

df_connectivity = (
    airport_count
    .merge(routes_by_country, on="Country", how="left")
    .merge(international_reach, on="Country", how="left")
    .merge(hub_strength, on="Country", how="left")
)

In [7]:
df_connectivity = df_connectivity.fillna(0)

In [8]:
# Normalize (min–max scaling)
for col in ["airport_count", "total_routes", "unique_dest_countries", "hub_strength"]:
    df_connectivity[col + "_scaled"] = (df_connectivity[col] - df_connectivity[col].min()) / (df_connectivity[col].max() - df_connectivity[col].min())

# Compute Connectivity Index

# Weighted formula:

df_connectivity["connectivity_index"] = (
    0.25 * df_connectivity["airport_count_scaled"] +
    0.35 * df_connectivity["total_routes_scaled"] +
    0.30 * df_connectivity["unique_dest_countries_scaled"] +
    0.10 * df_connectivity["hub_strength_scaled"]
)

# Final output
connectivity_index = df_connectivity[["Country", "connectivity_index"]]
connectivity_index.sort_values("connectivity_index", ascending=False).head(20)

,Country,connectivity_index
223,United States,0.849593
43,China,0.428628
73,France,0.414453
79,Germany,0.378047
222,United Kingdom,0.376665
149,Netherlands,0.310143
215,Turkey,0.309927
221,United Arab Emirates,0.307507
37,Canada,0.307250
176,Russia,0.292943


In [9]:
df_connectivity.to_csv("connectivity_index.csv", index=False)

In [30]:
# -------------------------
# 1. Load datasets
# -------------------------

stadiums = pd.read_csv("/content/drive/MyDrive/DATA 6560/Data/Raw datasets/all_stadiums.csv")
engagement = pd.read_csv("/content/drive/MyDrive/DATA 6560/Data/Raw datasets/fan_engagement.csv")
gdp = pd.read_csv(
    "/content/drive/MyDrive/DATA 6560/Data/Raw datasets/gdp_pop_urban.csv",
    on_bad_lines='skip'  # skips malformed rows
)
connect = pd.read_csv("/content/drive/MyDrive/DATA 6560/Data/Raw datasets/connectivity_index.csv")

# Standardize column names
stadiums.rename(columns={"country": "Country"}, inplace=True)
engagement.rename(columns={"country": "Country", "iso3": "ISO2"}, inplace=True)
gdp.rename(columns={"country": "Country", "iso2": "ISO2"}, inplace=True)
connect.rename(columns={"Country": "Country"}, inplace=True)

In [32]:
# -------------------------
# 2. Prep datasets
# -------------------------

# A. Standardize country names

country_corrections = {
    "Portuguese Republic": "Portugal",
    "England": "United Kingdom",
    "Korea": "South Korea",
    "French Republic": "France",
    "Czech Republic": "Czechia",
    "Italian Republic": "Italy",
    "Hellenic Republic": "Greece",
    "Argentine Republic": "Argentina",
    "United Mexican States": "Mexico"
}

# Apply corrections
stadiums['Country'] = stadiums['Country'].replace(country_corrections)

# B.Remove commas and convert to numeric, coercing errors to NaN
stadiums['total_capacity'] = pd.to_numeric(
    stadiums['total_capacity'].astype(str).str.replace(',', '').str.strip(),
    errors='coerce'
)

# Now aggregate
stadiums_agg = (
    stadiums.groupby("Country")
            .agg(
                stadium_count=("id", "count"),
                total_stadium_capacity=("total_capacity", "sum")
             )
             .reset_index() )

# B. Aggregate stadium capacity by country

stadiums_agg = (
    stadiums.groupby("Country")
            .agg(
                stadium_count=("id", "count"),
                total_stadium_capacity=("total_capacity", "sum")
             )
             .reset_index())
# Compute average stadium capacity
stadiums_agg['average_stadium_capacity'] = (
    stadiums_agg['total_stadium_capacity'] / stadiums_agg['stadium_count']
).round(0).astype('Int64')

# Round to nearest integer
stadiums_agg['average_stadium_capacity'] = stadiums_agg['average_stadium_capacity'].round(0).astype('Int64')

# Compute averages per country while keeping ISO2
gdp_avg = gdp.groupby(["Country", "ISO2"]).agg(
    gdp_per_capita=("gdp_per_capita", "mean"),
    population=("population", "mean"),
    urban_pct=("urban_pct", "mean")
).reset_index()

# -------------------------
# 3. Merge using ISO2 when available
# -------------------------

# Start with fan engagement (core table)
df = engagement.copy()

# Merge GDP
df = df.merge(
    gdp_avg[["ISO2", "gdp_per_capita", "population", "urban_pct"]],
    on="ISO2",
    how="left"
)

# Merge stadium aggregates using Country name
df = df.merge(
    stadiums_agg,
    on="Country",
    how="left"
)

# Merge connectivity index (country-based)
df = df.merge(
    connect,
    on="Country",
    how="left"
)

# -------------------------
# 4. Fill missing numeric data with 0 where appropriate
# -------------------------

numeric_cols = [
    "gdp_per_capita", "population", "urban_pct",
    "stadium_count", "total_stadium_capacity",
    "airport_count", "total_routes", "unique_dest_countries",
    "hub_strength", "airport_count_scaled",
    "total_routes_scaled", "unique_dest_countries_scaled",
    "hub_strength_scaled", "connectivity_index"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Move ISO2 to the first column
cols = ['ISO2'] + [c for c in df.columns if c != 'ISO2']
df = df[cols]

In [33]:
df.to_csv("merged_data.csv", index=False)

In [35]:
# ------------------------------
# GATE 1 — Record counts
# ------------------------------
print("Gate 1 — Record Counts")
print("-----------------------")

total_rows = len(df)
missing_iso2 = df["ISO2"].isna().sum()

print(f"Total countries: {total_rows}")
print(f"Countries missing ISO2: {missing_iso2}")
assert total_rows > 100, "FAIL: Not enough countries — suspicious dataset size."
print("PASS: Sufficient country coverage.\n")


# ------------------------------
# GATE 2 — Key uniqueness
# ------------------------------
print("Gate 2 — Key Uniqueness")
print("-------------------------")

dup_keys = df.duplicated(subset=["ISO2"]).sum()
print(f"Duplicate ISO2 entries: {dup_keys}")
assert dup_keys == 0, "FAIL: Duplicate ISO2 values detected — merge error."
print("PASS: ISO2 is unique.\n")


# ------------------------------
# GATE 3 — Range checks
# ------------------------------
print("Gate 3 — Range Checks")
print("-----------------------")

range_checks = {
    "gdp_per_capita": (0, 500000),
    "population": (0, 2e9),
    "urban_pct": (0, 100),
    "gt_composite": (0, 100),
    "stadium_count": (0, 1000),
    "average_stadium_capacity": (0, 100000),
    "total_stadium_capacity": (0, 2e7),
    "connectivity_index": (0, 1),
}

for col, (min_val, max_val) in range_checks.items():
    if col in df.columns:
        min_actual = df[col].min()
        max_actual = df[col].max()
        print(f"{col}: min={min_actual}, max={max_actual}")

        assert min_actual >= min_val - 1e-9, f"FAIL: {col} min value out of range."
        assert max_actual <= max_val + 1e-9, f"FAIL: {col} max value out of range."

print("PASS: Numeric ranges valid.\n")


# ------------------------------
# GATE 4 — Logic checks
# ------------------------------
print("Gate 4 — Logic Checks")
print("-----------------------")

# 4.1 Urban_pct > 0 only when population > 0
invalid_urban = df[(df["urban_pct"] > 0) & (df["population"] == 0)]
assert len(invalid_urban) == 0, "FAIL: Countries have urban_pct but zero population."
print("PASS: Urbanization logical.\n")

# 4.2 Connectivity index should follow scaling between 0–1
assert df["connectivity_index"].between(0,1).all(), "FAIL: Connectivity index outside [0,1]."
print("PASS: Connectivity index scaled correctly.\n")

# 4.3 Stadium counts or capacities should be 0 for non-sporting microstates or missing data
high_capacity_no_stadiums = df[(df["stadium_count"] == 0) & (df["total_stadium_capacity"] > 0)]
assert len(high_capacity_no_stadiums) == 0, "FAIL: Capacity > 0 but stadium_count = 0."
print("PASS: Stadium logic consistent.\n")


# ------------------------------
# GATE 5 — Fan engagement logic
# ------------------------------
print("Gate 5 — Engagement Logic")
print("--------------------------")

for col in ["gt_NFL", "gt_American_football", "gt_Super_Bowl", "gt_composite"]:
    if col in df.columns:
        assert df[col].between(0, 100).all(), f"FAIL: {col} outside 0–100 Google Trends range."

print("PASS: Engagement scores valid.\n")


# ------------------------------
# Final PASS
# ------------------------------
print("============================")
print(" ALL VALIDATION GATES PASSED")
print("============================\n")


Gate 1 — Record Counts
-----------------------
Total countries: 182
Countries missing ISO2: 1
PASS: Sufficient country coverage.

Gate 2 — Key Uniqueness
-------------------------
Duplicate ISO2 entries: 0
PASS: ISO2 is unique.

Gate 3 — Range Checks
-----------------------
gdp_per_capita: min=0.0, max=215418.91063481523
population: min=0.0, max=1420046750.3333333
urban_pct: min=0.0, max=100.0
gt_composite: min=0.0, max=69.66666666666667
stadium_count: min=0.0, max=371.0
average_stadium_capacity: min=10000, max=65000
total_stadium_capacity: min=0.0, max=11775000.0
connectivity_index: min=0.0, max=0.8495934284766014
PASS: Numeric ranges valid.

Gate 4 — Logic Checks
-----------------------
PASS: Urbanization logical.

PASS: Connectivity index scaled correctly.

PASS: Stadium logic consistent.

Gate 5 — Engagement Logic
--------------------------
PASS: Engagement scores valid.

 ALL VALIDATION GATES PASSED

